## Demonstrate interactions with relations using omnikeeper

In [1]:
import os
import omnikeeper_client as okc
from omnikeeper_client import TraitDefinition, TraitAttributeDefinition, TraitRelationDefinition, ATTRIBUTETYPE_TEXT, ATTRIBUTETYPE_INTEGER
import pandas as pd

As usual create the omnikeeper client

In [2]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

Define the trait names and layer

In [3]:
host_trait_name = "python_client_demo.host"
interface_trait_name = "python_client_demo.interface"
layer_name = "relations"

Make sure that traits are created correctly

In [4]:
is_host_trait_created = okc.upsert_trait(okapiclient, TraitDefinition(host_trait_name, [
        TraitAttributeDefinition("id", "host.id", ATTRIBUTETYPE_INTEGER),
      ], optional_relations = [
        TraitRelationDefinition(identifier = "has_interface", predicate_id = "has_interface", direction_forward=True)
      ]))
print(is_host_trait_created)

is_interface_trait_created = okc.upsert_trait(okapiclient, TraitDefinition(interface_trait_name, [
        TraitAttributeDefinition("id", "interface.id", ATTRIBUTETYPE_INTEGER),
      ], optional_relations = [
        TraitRelationDefinition(identifier = "has_host", predicate_id = "has_host", direction_forward=True)
      ]))
print(is_interface_trait_created)

True
True


Create the host and interface CIs

In [5]:
host_ci_id = okc.create_ci(okapiclient, "Host A", layer_name)
host_ci_id

UUID('92196245-6679-4295-b1dd-fb655286a5a4')

Update created host ci, add host.id attribute

In [6]:
host_attributes = {
    "host.id": 1,
}

okc.mutate_ci(okapiclient, layer_name, host_ci_id, host_attributes)

{'mutateCIs': {'affectedCIs': [{'id': '92196245-6679-4295-b1dd-fb655286a5a4'}]}}

In [7]:
interface_ci_id = okc.create_ci(okapiclient, "Interface A", layer_name)
interface_ci_id

UUID('cca0feea-986e-4ca9-90f4-e4e0575ddb96')

In [8]:
interface_attributes = {
    "interface.id": 1,
}

okc.mutate_ci(okapiclient, layer_name, interface_ci_id, interface_attributes)

{'mutateCIs': {'affectedCIs': [{'id': 'cca0feea-986e-4ca9-90f4-e4e0575ddb96'}]}}

Create another interface ci

In [9]:
interface_ci_id_B = okc.create_ci(okapiclient, "Interface B", layer_name)
interface_ci_id_B
interface_attributes_B = {
    "interface.id": 2,
}
okc.mutate_ci(okapiclient, layer_name, interface_ci_id_B, interface_attributes_B)

{'mutateCIs': {'affectedCIs': [{'id': '011f341e-7aff-43e6-82f4-1b2ed82b8e9e'}]}}

Add host and interface relations using the created cis and okc

In [10]:
okc.add_trait_relations_by_ciid(okapiclient, host_trait_name, "has_interface", host_ci_id, [interface_ci_id, interface_ci_id_B], layer_name)

True

In [11]:
okc.add_trait_relations_by_ciid(okapiclient, interface_trait_name, "has_host", interface_ci_id, [host_ci_id], layer_name)
okc.add_trait_relations_by_ciid(okapiclient, interface_trait_name, "has_host", interface_ci_id_B, [host_ci_id], layer_name)

True

### Relations operations using lists

Fetch hosts and interfaces from the specified layer

In [12]:
hosts = okc.get_all_traitentities_list(okapiclient, trait_name=host_trait_name, layers=[layer_name])
hosts

[{'ciid': '92196245-6679-4295-b1dd-fb655286a5a4', 'id': 1}]

In [13]:
interfaces = okc.get_all_traitentities_list(okapiclient, trait_name=interface_trait_name, layers=[layer_name])
interfaces

[{'ciid': 'cca0feea-986e-4ca9-90f4-e4e0575ddb96', 'id': 1},
 {'ciid': '011f341e-7aff-43e6-82f4-1b2ed82b8e9e', 'id': 2}]

Fetch has_host realtions. In the result interfaces will be as base_cis and hosts as related cis

In [14]:
has_host_relations = okc.get_trait_relation_list(okapiclient, trait_name=interface_trait_name, relation_name="has_host", layers=[layer_name])
has_host_relations

[{'base_ciid': 'cca0feea-986e-4ca9-90f4-e4e0575ddb96',
  'related_ciids': ['92196245-6679-4295-b1dd-fb655286a5a4']},
 {'base_ciid': '011f341e-7aff-43e6-82f4-1b2ed82b8e9e',
  'related_ciids': ['92196245-6679-4295-b1dd-fb655286a5a4']}]

Fetch has_interface relations for all hosts

In [15]:
has_interface_relations = okc.get_trait_relation_list(okapiclient, trait_name=host_trait_name, relation_name="has_interface", layers=[layer_name])
has_interface_relations

[{'base_ciid': '92196245-6679-4295-b1dd-fb655286a5a4',
  'related_ciids': ['cca0feea-986e-4ca9-90f4-e4e0575ddb96',
   '011f341e-7aff-43e6-82f4-1b2ed82b8e9e']}]

Merge hosts with realted cis in this case interfaces

In [16]:
interface_dict = {interface['ciid']: interface for interface in interfaces}

hosts_with_relations = []
for host in hosts:
    merged_dict = host.copy()
    merged_dict['related_ciis'] = []
    for relation in has_interface_relations:
        if host['ciid'] == relation['base_ciid']:
            for related_ciid in relation['related_ciids']:
                if related_ciid in interface_dict:
                    merged_dict['related_ciis'].append(interface_dict[related_ciid])
    hosts_with_relations.append(merged_dict)

hosts_with_relations

[{'ciid': '92196245-6679-4295-b1dd-fb655286a5a4',
  'id': 1,
  'related_ciis': [{'ciid': 'cca0feea-986e-4ca9-90f4-e4e0575ddb96', 'id': 1},
   {'ciid': '011f341e-7aff-43e6-82f4-1b2ed82b8e9e', 'id': 2}]}]

The same can be done for interfaces to fetch its relations

In [17]:
host_dict = {host['ciid']: host for host in hosts}

interfaces_with_relations = []
for interface in interfaces:
    merged_dict = interface.copy()
    merged_dict['related_ciis'] = []
    for relation in has_host_relations:
        if interface['ciid'] == relation['base_ciid']:
            for related_ciid in relation['related_ciids']:
                if related_ciid in host_dict:
                    merged_dict['related_ciis'].append(host_dict[related_ciid])
    interfaces_with_relations.append(merged_dict)

interfaces_with_relations

[{'ciid': 'cca0feea-986e-4ca9-90f4-e4e0575ddb96',
  'id': 1,
  'related_ciis': [{'ciid': '92196245-6679-4295-b1dd-fb655286a5a4', 'id': 1}]},
 {'ciid': '011f341e-7aff-43e6-82f4-1b2ed82b8e9e',
  'id': 2,
  'related_ciis': [{'ciid': '92196245-6679-4295-b1dd-fb655286a5a4', 'id': 1}]}]

### Relations operations using pandas dataframes

We will do the same steps here but using pandas dataframes to work with omnikeeper data

In [18]:
has_host_relations_df = okc.get_trait_relation_dataframe(okapiclient, trait_name=interface_trait_name, relation_name="has_host", layers=[layer_name])
has_interface_relations_df = okc.get_trait_relation_dataframe(okapiclient, trait_name=host_trait_name, relation_name="has_interface", layers=[layer_name])
has_interface_relations_df

,related_ciids
base_ciid,
92196245-6679-4295-b1dd-fb655286a5a4,"[cca0feea-986e-4ca9-90f4-e4e0575ddb96, 011f341..."


Fetch hosts and interfaces using dataframes 

In [19]:
hosts_df = okc.get_all_traitentities_dataframe(okapiclient, trait_name=host_trait_name, layers=[layer_name])
interfaces_df = okc.get_all_traitentities_dataframe(okapiclient, trait_name=interface_trait_name, layers=[layer_name])
hosts_df

,id
ciid,
92196245-6679-4295-b1dd-fb655286a5a4,1


Now merge the related data. In this case we can see how easy it is to merge related data using pandas dataframes

In [20]:
hosts_with_relations_df = hosts_df.merge(has_interface_relations_df, left_index=True, right_index=True, how='outer')
hosts_with_relations_df

,id,related_ciids
ciid,,
92196245-6679-4295-b1dd-fb655286a5a4,1,"[cca0feea-986e-4ca9-90f4-e4e0575ddb96, 011f341..."


We can create a multi index dataframe which includes interfaces information

In [21]:
df_exploded = hosts_with_relations_df.explode('related_ciids').reset_index()
merged_df = pd.merge(df_exploded, interfaces_df, left_on='related_ciids', right_on='ciid', how='left', suffixes=('_host', '_interface'))
merged_df.set_index(['ciid', 'related_ciids'], inplace=True)
merged_df

id_host  \
ciid                                 related_ciids                                   
92196245-6679-4295-b1dd-fb655286a5a4 cca0feea-986e-4ca9-90f4-e4e0575ddb96        1   
                                     011f341e-7aff-43e6-82f4-1b2ed82b8e9e        1   

                                                                           id_interface  
ciid                                 related_ciids                                       
92196245-6679-4295-b1dd-fb655286a5a4 cca0feea-986e-4ca9-90f4-e4e0575ddb96             1  
                                     011f341e-7aff-43e6-82f4-1b2ed82b8e9e             2